In [ ]:
# IMPORTS
from tensorflow.keras import layers, models
import time
import ast
import tensorflow as tf
from copy import deepcopy
import random
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# GLOBAL VARIABLES
show = True
predictor_data_filename = "/home/data.txt"
input_size = 18

In [ ]:
# PREDICTOR MODEL
class Predictor:
    def __init__(self, n_layers, keras_loss_fun, output_activation, hyperparameters):
        # Create input and hidden layers
        input_layer = layers.Input(shape=(input_size,))
        last_layer = input_layer
        for i in range(0, n_layers):
            last_layer = layers.Dense(hyperparameters[i]['size'], name="dense_layer_{}".format(i), activation=hyperparameters[i]['activation'])(last_layer)

        # Create output layers (accuracy, test_latency)
        output_layer_cnn_accuracy = layers.Dense(1, name="output_layer_cnn_accuracy", activation=output_activation)(last_layer)
        output_layer_cnn_latency = layers.Dense(1, name="output_layer_cnn_latency")(last_layer)

        model = models.Model(inputs=input_layer, outputs=[output_layer_cnn_accuracy, output_layer_cnn_latency])

        model.compile(optimizer='adam',
                      loss=keras_loss_fun,
                      metrics=['mae'])

        self.model = model

    def train(self, train_data, train_labels, test_data, test_labels):

        history = self.model.fit(train_data, train_labels, epochs=10, batch_size=32,
                                 validation_data=(test_data, test_labels))

        start_time = time.time()
        loss, output_layer_cnn_accuracy_loss, output_layer_cnn_latency_loss, output_layer_cnn_accuracy_mae, output_layer_cnn_latency_mae = self.model.evaluate(test_data, test_labels)
        end_time = time.time()

        test_time = end_time - start_time

        return output_layer_cnn_accuracy_loss, output_layer_cnn_latency_loss, test_time

In [ ]:
# TRAIN AND TEST DATA
def _get_predictor_data(filename):
    hyperparameters, outputs = _parse_predictor_data_file(filename)
    hyperparameters, outputs = _preprocess_predictor_data(hyperparameters, outputs)

    return train_test_split(hyperparameters, outputs, test_size=0.2)


activation_mappings = {
    'linear': 0,
    'relu': 1,
    'sigmoid': 5,
    'tanh': 6,
    'softmax': 8,
    'exponential': 10,
}

padding_mappings = {
    'valid': 0,
    'same': 1,
}


def _preprocess_predictor_data(inputs, outputs):
    processed_inputs = []
    for model in inputs:
        processed_model = []
        for layer_type, params in model.items():  # Iterate over all layers
            if 'conv' in layer_type:
                processed_model.append(activation_mappings[params['activation']])
                processed_model.append(padding_mappings[params['padding']])
                processed_model.append(params['kernel_size'])
                processed_model.append(params['filters'] / 10)  #  Make it around the same size as the others
            else:
                processed_model.append(padding_mappings[params['padding']])
                processed_model.append(params['strides'])
                processed_model.append(params['pool_size'])
        if show:
            print("\nModel = {}".format(model))
            print("\n--> Processed model = {}\n".format(processed_model))
        processed_inputs.append(processed_model)
    return np.array(processed_inputs), np.array(outputs)


def _parse_predictor_data_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    hyperparameters = []
    outputs = []  # Output  = (accuracy, latency)
    i = 0
    for line in lines:
        i += 1
        line = line.strip()
        try:
          result = ast.literal_eval(line)
          hyperparameters.append(result['HP'])
          outputs.append((result['Accuracy'], result['Latency: ']))
        except:
           if show:
                print("Error parsing data line ({})\n".format(i))

    return hyperparameters, outputs



# GET PREDICTOR DATA
training_data, testing_data, training_labels, testing_labels = _get_predictor_data(predictor_data_filename)

Error parsing data line (400)

Error parsing data line (542)

Error parsing data line (624)

Error parsing data line (633)

Error parsing data line (724)


Model = {'conv_1': {'activation': 'relu', 'kernel_size': 3, 'padding': 'valid', 'filters': 32}, 'pool_1': {'pool_size': 2, 'strides': 2, 'padding': 'valid'}, 'conv_2': {'activation': 'relu', 'kernel_size': 3, 'padding': 'valid', 'filters': 64}, 'pool_2': {'pool_size': 2, 'strides': 2, 'padding': 'valid'}, 'conv_3': {'activation': 'relu', 'kernel_size': 3, 'padding': 'valid', 'filters': 64}}

--> Processed model = [1, 0, 3, 3.2, 0, 2, 2, 1, 0, 3, 6.4, 0, 2, 2, 1, 0, 3, 6.4]


Model = {'conv_1': {'activation': 'tanh', 'kernel_size': 3, 'padding': 'valid', 'filters': 13}, 'pool_1': {'pool_size': 2, 'strides': 2, 'padding': 'valid'}, 'conv_2': {'activation': 'relu', 'kernel_size': 3, 'padding': 'same', 'filters': 92}, 'pool_2': {'pool_size': 2, 'strides': 2, 'padding': 'valid'}, 'conv_3': {'activation': 'relu', 'kernel_size': 4, 'paddin

In [ ]:
# PREDICTOR HELPER

predictor_loss_set = [
    'mean_squared_error',
    'mean_absolute_error',
    tf.keras.losses.Huber(),
    tf.keras.losses.LogCosh()
]


predictor_output_activation_set = ['sigmoid', 'linear']

predictor_n_layer_set = range(20, 30)

predictor_layer_hp_set = {
    'activation': [
        'relu',
        'selu',
        'tanh',
        'linear',
        'swish',
    ],
    'size': [20, 40, 60, 80, 100]
}


def get_predictor_layer_default_hyperparameters():
    return {
        'activation': random.sample(predictor_layer_hp_set['activation'], 1)[0],
        'size': random.sample(predictor_layer_hp_set['size'], 1)[0]
    }


def get_default_predictor(n_layers, loss, output_activation):
    return {
        "keras_loss_fun": loss,
        "n_layers": n_layers,
        "output_activation": output_activation,
        "hyperparameters": [get_predictor_layer_default_hyperparameters() for _ in range(0, n_layers)]
    }

def predictor_objective(params):
    n_layers = params["n_layers"]
    keras_loss_fun = params["keras_loss_fun"]
    output_activation = params["output_activation"]
    hyperparameters = params["hyperparameters"]
    predictor = Predictor(n_layers, keras_loss_fun, output_activation, hyperparameters)

    cnn_acc_loss, cnn_latency_loss, test_time = predictor.train(training_data, training_labels, testing_data, testing_labels)

    obj_value = 1 / test_time * (cnn_acc_loss * cnn_latency_loss)**2
    if show:
        print("\n", "-" * 8, "params:  {}".format(params), "-" * 8)
        print("\n", "-" * 8, "#### test_accuracy_loss: {} #### test_latency_loss: {} #### test_time: {} #### objective: {}"
            .format(cnn_acc_loss, cnn_latency_loss,test_time, obj_value), "-" * 8)

    return obj_value


def predictor_get_random_neighbouring_solution(old_solution):
    solution = deepcopy(old_solution)
    n_layers = solution["n_layers"]

    # Change the size of two layers
    change_size_layers = random.sample(range(0, n_layers), 2)
    for change_size_layer in change_size_layers:
        new_size = random.sample(predictor_layer_hp_set["size"], 1)[0]
        solution["hyperparameters"][change_size_layer]["size"] = new_size
    # Change activation of one layer
    change_activation_layer = random.sample(range(0, n_layers), 1)[0]
    new_activation = random.sample(predictor_layer_hp_set["activation"], 1)[0]
    solution["hyperparameters"][change_activation_layer]["activation"] = new_activation

    return solution


In [ ]:
# METAHEURISTICS: SIMULATED ANNEALING
def simulated_annealing(initial_solution, neighbour_gen_fun, objective_fun, iterations, c, seed=-1):
    # Initialization phase
    if seed >= 0:
        random.seed = seed
    current_solution = initial_solution
    current_eval = objective_fun(current_solution)
    # Optimization phase
    for i in range(iterations):
        next_solution = neighbour_gen_fun(current_solution)
        next_eval = objective_fun(next_solution)
        if next_eval > current_eval or current_eval == 0 or _accept_worse_solution(c, i+1, current_eval, next_eval):
            current_solution = next_solution
            current_eval = next_eval
            if show:
                print("\n", "#", "Accepted new solution")
    return current_solution, current_eval


def _accept_worse_solution(c, interval, curr_val, next_val):
    return random.random() < np.exp(interval * (next_val - curr_val) / (c * curr_val))

In [ ]:
# OPTIMIZER

def _predictor_sa_optimization(n_layers, loss_fun, output_activation):
    c = 1
    iterations_per_conf = 50
    seed = -1
    return simulated_annealing(get_default_predictor(n_layers, loss_fun, output_activation),
                               predictor_get_random_neighbouring_solution,
                               predictor_objective,
                               iterations_per_conf,
                               c,
                               seed)

def optimize_predictor(optimize_fun):
    best_results = {}

    for output_activation in predictor_output_activation_set:
      for n_layers in predictor_n_layer_set:
          for loss_fun in predictor_loss_set:
              model, eval = optimize_fun(n_layers, loss_fun, output_activation)
              print("#### LAYERS: {}, LOSS: {}: MODEL: {}, EVAL: {} ####\n".format(n_layers, loss_fun, model, eval))
              best_results[n_layers, loss_fun] = {
                  "model": model,
                  "eval": eval,
              }


In [ ]:
# RUN
optimize_predictor(_predictor_sa_optimization)

Epoch 1/10
21/21 [==============================] - 5s 42ms/step - loss: 1.3076 - output_layer_cnn_accuracy_loss: 0.6459 - output_layer_cnn_latency_loss: 0.6617 - output_layer_cnn_accuracy_mae: 0.6608 - output_layer_cnn_latency_mae: 0.6781 - val_loss: 0.9370 - val_output_layer_cnn_accuracy_loss: 0.4700 - val_output_layer_cnn_latency_loss: 0.4670 - val_output_layer_cnn_accuracy_mae: 0.6118 - val_output_layer_cnn_latency_mae: 0.6118
Epoch 2/10
21/21 [==============================] - 0s 12ms/step - loss: 1.0276 - output_layer_cnn_accuracy_loss: 0.5195 - output_layer_cnn_latency_loss: 0.5081 - output_layer_cnn_accuracy_mae: 0.6346 - output_layer_cnn_latency_mae: 0.6346 - val_loss: 0.9176 - val_output_layer_cnn_accuracy_loss: 0.4660 - val_output_layer_cnn_latency_loss: 0.4516 - val_output_layer_cnn_accuracy_mae: 0.6118 - val_output_layer_cnn_latency_mae: 0.6118
Epoch 3/10
21/21 [==============================] - 0s 14ms/step - loss: 1.0198 - output_layer_cnn_accuracy_loss: 0.5180 - output_

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7de6f075bc70>
Traceback (most recent call last):
  File "/usr/lib/python3.10/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 
Exception ignored in: <function _xla_gc_callback at 0x7de6b19363b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 101, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


21/21 [==============================] - 37s 2s/step - loss: 1.4902 - output_layer_cnn_accuracy_loss: 0.7277 - output_layer_cnn_latency_loss: 0.7626 - output_layer_cnn_accuracy_mae: 0.6767 - output_layer_cnn_latency_mae: 0.7173 - val_loss: 0.9428 - val_output_layer_cnn_accuracy_loss: 0.4847 - val_output_layer_cnn_latency_loss: 0.4581 - val_output_layer_cnn_accuracy_mae: 0.6118 - val_output_layer_cnn_latency_mae: 0.6118
Epoch 2/10
21/21 [==============================] - 0s 11ms/step - loss: 1.0310 - output_layer_cnn_accuracy_loss: 0.5216 - output_layer_cnn_latency_loss: 0.5094 - output_layer_cnn_accuracy_mae: 0.6346 - output_layer_cnn_latency_mae: 0.6346 - val_loss: 0.9222 - val_output_layer_cnn_accuracy_loss: 0.4654 - val_output_layer_cnn_latency_loss: 0.4567 - val_output_layer_cnn_accuracy_mae: 0.6118 - val_output_layer_cnn_latency_mae: 0.6118
Epoch 3/10
21/21 [==============================] - 0s 11ms/step - loss: 1.0239 - output_layer_cnn_accuracy_loss: 0.5174 - output_layer_cnn_la

KeyboardInterrupt: ignored